In [1]:
auth = '757F054AA22E1711B598A89EB276EDC193933EC8'

list_appid = ['tfy6d12dc-19a5-4093-9f00-f276d950f77',
              'picheng11-5163-4258-be21-a5629527a07',
              'Shanghai-b2aa-4eed-82c3-b1e42e2e9179',
              'danjidin-f8b1-4e84-adce-026944025ceb',
              'ibay365a-b93e-46e7-8b46-6a33b716ab2a',
              '-IBAY365-PRD-08e35c535-85b731b3']

In [2]:
import pandas as pd

In [3]:
df_menu = pd.read_excel('./menu.xlsx', header=0, dtype={'Item_Number': str})
list_item = df_menu['Item_Number'].to_list()

print('总数量：' + str(len(list_item)))
list_item

FileNotFoundError: [Errno 2] No such file or directory: './menu.xlsx'

In [6]:
df = pd.DataFrame(columns=['Item_Number',
                           'Title',
                           'Price',
                           'Sold',
                           'Vehicle',
                           'Src',
                           'status'])

df

,Item_Number,Title,Price,Sold,Vehicle,Src,status


In [7]:
import random
import requests
import json

In [8]:
i = 0

index_appid = random.randint(0, 5)

developer = 'http://106.75.138.2/index.php/api/GetAppToken?auth=' + auth + '&appid=' + list_appid[index_appid]
resp = requests.get(developer).text
token = json.loads(resp)['getapptokenResponse']['token']

token

'v^1.1#i^1#I^3#r^0#f^0#p^1#t^H4sIAAAAAAAAAOVYbWwURRju9QMFLA1KENHAsWAjmN2bvb3e3q29g20Lcha4wrX1KFGY3Z3trd3bXXb22l6jyVEMChJU4AeREhDURNT4FepHIjEYMCokEIwxQYwkCDXRCIQEEwhx93qUayWA3MU08f5c5p133nmeZ953ZnZAZszYOesWrrtU6bqrdFcGZEpdLno8GDum4tEJZaVTK0pAnoNrV2ZWpry3bKAWw6RqcMsQNnQNI3d3UtUwlzWGiJSpcTrECuY0mESYs0Quxi9exHkpwBmmbumirhLuSEOIQBBBhqFZyNIISoCxrdq1mM16iKBpwc8yguj3MTAoyYLdj3EKRTRsQc0KEV7gZUjAknSgGbCcD3DARwXZQBvhbkUmVnTNdqEAEc7C5bJjzTysN4cKMUamZQchwhF+QSzKRxrmL2mu9eTFCud0iFnQSuHhrXpdQu5WqKbQzafBWW8ulhJFhDHhCQ/OMDwox18Dcwfws1IHadHHMKLoZSGSJVYqipQLdDMJrZvjcCyKRMpZVw5plmKlb6WorYbwDBKtXGuJHSLS4Hb+lqagqsgKMkPE/Dp+Od/URIRjCai1J6BCCl4ISR9CEhnwigwp0MjnRV4UpNlgbpLBSDmJR8xSr2uS4giG3Ut0qw7ZiNFIXZg8XWynqBY1edly0OT7BYf0Y9qcBR1cwZSV0Jw1RUlbBHe2eWv1h0ZblqkIKQsNRRjZkZUnREDDUCRiZGc2D3Op041DRMKyDM7j6erqoroYSjfbPV4AaE988aKYmEBJSOR8nVrvxsqtB5BKloqI7JFY4ay0YWPptvPUBqC1E+Ggj2VrcrIPRxUeaf2HIY+yZ3gxFKs4ZK/Py0oiCrACA6FQlH0mnMtPj4MDCTBNJqHZgSxDhSIiRTvNUklkKhLH1MheJiAjUvIHZdIXlGVSqJH8JC0jBBASBDEY+L/UyO1meQyJJrKKluZFSfE

In [9]:
headers = {'X-EBAY-API-IAF-TOKEN': token}
resp = requests.get('https://open.api.ebay.com/shopping?callname=GetSingleItem&appid=' + list_appid[index_appid] + '&responseencoding=JSON&siteid=0&version=967&IncludeSelector=Details,Compatibility,ItemSpecifics&ItemID=' + list_item[i], headers=headers).text

info = json.loads(resp)
if info['Ack'] != 'Success':
    print('Error')
else:
    print('Success')
    
info

Success


{'Timestamp': '2023-07-18T08:21:46.377Z',
 'Ack': 'Success',
 'Build': '1299_CORE_APILW_19146596_R1',
 'Version': '1299',
 'Item': {'BestOfferEnabled': True,
  'ItemID': '265482076619',
  'EndTime': '2023-08-03T07:38:05.000Z',
  'StartTime': '2022-01-03T08:38:05.000Z',
  'ViewItemURLForNaturalSearch': 'https://www.ebay.com/itm/Ventildeckel-Zylinderkopfhaube-fur-Citroen-C4-Peugeot-207-308-508-5008-1-6L-/265482076619',
  'ListingType': 'FixedPriceItem',
  'Location': 'Sülzetal',
  'PaymentMethods': [],
  'GalleryURL': 'https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/2YkAAOSws4Nh0mEi/$_12.JPG',
  'PictureURL': ['https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/2YkAAOSws4Nh0mEi/$_12.JPG',
   'https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/6mQAAOSws3ph0mED/$_12.JPG',
   'https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/a8MAAOSwXjlh0mEH/$_12.JPG',
   'https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/AmwAAOSwvsth0mEK/$_12.JPG',
   'https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/GBQAAOSwleph0mEY/$_12.JPG',
   'https://i.ebayimg.c

In [10]:
# Item.Title - None
title = info['Item']['Title']

title

'Ventildeckel Zylinderkopfhaube für Citroën C4 Peugeot 207 308 508 5008 1.6L'

In [11]:
# Item.CurrentPrice - None
price = info['Item']['CurrentPrice']['CurrencyID'] + ' ' + str(info['Item']['CurrentPrice']['Value'])

price

'EUR 74.99'

In [12]:
# Item.QuantitySold - Details
sold = info['Item']['QuantitySold']

sold

21

In [13]:
# Item.ItemCompatibilityList - Compatibility
if 'ItemCompatibilityList' in info['Item']:
    vehicle = info['Item']['ItemCompatibilityList']['Compatibility']
else:
    vehicle = ''
    
vehicle

[{'NameValueList': [{'Value': []},
   {'Name': 'Make', 'Value': ['Citroën']},
   {'Name': 'Model', 'Value': ['C4 Coupe']},
   {'Name': 'Platform', 'Value': ['LA']},
   {'Name': 'Type', 'Value': ['1.6 THP']},
   {'Name': 'Production Period', 'Value': ['2008/06-2011/07']},
   {'Name': 'Engine', 'Value': ['1598 ccm, 103 KW, 140 PS']}],
  'CompatibilityNotes': ''},
 {'NameValueList': [{'Value': []},
   {'Name': 'Make', 'Value': ['Citroën']},
   {'Name': 'Model', 'Value': ['C4 Coupe']},
   {'Name': 'Platform', 'Value': ['LA']},
   {'Name': 'Type', 'Value': ['1.6 THP 150']},
   {'Name': 'Production Period', 'Value': ['2008/07-2011/07']},
   {'Name': 'Engine', 'Value': ['1598 ccm, 110 KW, 150 PS']}],
  'CompatibilityNotes': ''},
 {'NameValueList': [{'Value': []},
   {'Name': 'Make', 'Value': ['Citroën']},
   {'Name': 'Model', 'Value': ['C4 Grand Picasso I']},
   {'Name': 'Platform', 'Value': ['UA']},
   {'Name': 'Type', 'Value': ['1.6 16V']},
   {'Name': 'Production Period', 'Value': ['2008/0

In [14]:
dict_vehicle = {}
for list_name_value in vehicle:
    make = ''
    model = ''
    year = ''
    for name_value in list_name_value['NameValueList']:
        if 'Name' in name_value and 'Make' in name_value['Name']:
            make = name_value['Value'][0]
        elif 'Name' in name_value and 'Model' in name_value['Name']:
            model = name_value['Value'][0]
        elif 'Name' in name_value and 'Year' in name_value['Name']:
            year = name_value['Value'][0]
            
    if make not in dict_vehicle:
        dict_vehicle[make] = {model: [year]}
    else:
        if model not in dict_vehicle[make]:
            dict_vehicle[make][model] = [year]
        else:
            dict_vehicle[make][model].append(year)
            
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        list_vehicle.append(make + ' ' + model + ' ' + min(dict_vehicle[make][model]) + '-' + max(dict_vehicle[make][model]))
        
vehicle = '\n'.join(list_vehicle)

print(vehicle)

Citroën C4 Coupe -
Citroën C4 Grand Picasso I -
Citroën C4 Grand Picasso II -
Citroën C4 I -
Citroën C4 II -
Citroën C4 Picasso I Großraumlimousine -
Citroën C4 Picasso II -
Citroën C5 III -
Citroën C5 III Break -
Citroën DS3 -
Citroën DS3 Cabriolet -
Citroën DS4 -
Citroën DS5 -
Peugeot 207 -
Peugeot 207 CC -
Peugeot 207 SW -
Peugeot 208 I -
Peugeot 3008 Großraumlimousine -
Peugeot 308 CC -
Peugeot 308 I -
Peugeot 308 II -
Peugeot 308 SW I -
Peugeot 308 SW II -
Peugeot 5008 -
Peugeot 508 I -
Peugeot 508 SW I -
Peugeot RCZ -


In [15]:
# dict_vehicle = {}
# for name_value in vehicle:
#     make = name_value['NameValueList'][2]['Value'][0]
#     model = name_value['NameValueList'][3]['Value'][0]
#     try:
#         engine = re.findall(r'^[.0-9]+L', name_value['NameValueList'][5]['Value'][0])[0]
#     except:
#         continue
#     year = name_value['NameValueList'][1]['Value'][0]

#     if make not in dict_vehicle:
#         dict_vehicle[make] = {model: {engine: [year]}}
#     else:
#         if model not in dict_vehicle[make]:
#             dict_vehicle[make][model] = {engine: [year]}
#         else:
#             if engine not in dict_vehicle[make][model]:
#                 dict_vehicle[make][model][engine] = [year]
#             else:
#                 dict_vehicle[make][model][engine].append(year)

# list_vehicle = []
# for make in dict_vehicle:
#     for model in dict_vehicle[make]:
#         for engine in dict_vehicle[make][model]:
#             list_vehicle.append(make + ' ' + model + ' ' + engine + ' ' + min(dict_vehicle[make][model][engine]) + '-' + max(dict_vehicle[make][model][engine]))

# vehicle = '\n'.join(list_vehicle)

# print(vehicle)

In [16]:
# Item.GalleryURL - None
if 'GalleryURL' in info['Item']:
    src = info['Item']['GalleryURL']
elif 'PictureURL' in info['Item']:
    if len(info['Item']['PictureURL']) == 0:
        src = ''
    else:
        src = info['Item']['PictureURL'][0]
        
src

'https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/2YkAAOSws4Nh0mEi/$_12.JPG'

In [17]:
# Item.ItemSpecifics - ItemSpecifics
if 'ItemSpecifics' in info['Item']:
    details = info['Item']['ItemSpecifics']['NameValueList']
else:
    details = ''
    
details

[{'Name': 'Hersteller', 'Value': ['nicht zutreffend']},
 {'Name': 'Produktart',
  'Value': ['Motorventilabdeckung, Zylinderkopfabdeckung, Motorteile']},
 {'Name': 'Herstellernummer', 'Value': ['nicht zutreffend']},
 {'Name': 'OE/OEM Referenznummer(n)',
  'Value': ['9805712480', '9812071480', 'V759886280']},
 {'Name': 'Vergleichsnummer',
  'Value': ['Motorventildeckelsatz', 'Engine Valve Cover']},
 {'Name': 'Produkttyp',
  'Value': ['Zylinderkopfhaube, Rocker Cover, Ventildeckel Zylinderkopfhaube']},
 {'Name': 'Produktgruppe',
  'Value': ['Ventildeckeldichtung, Ventildeckel, Zylinderkopfhaube']},
 {'Name': 'Kompatibel für', 'Value': ['Citroën, Peugeot']},
 {'Name': 'für Modell-1',
  'Value': ['C4 Coupe, C4 Grand Picasso I, C4 Grand Picasso II, C4 I, C4 II']},
 {'Name': 'für Modell-2',
  'Value': ['C4 Picasso I, C4 Picasso II, C5 III, C5 III Break']},
 {'Name': 'für Modell-3',
  'Value': ['DS3, DS3 Cabriolet, DS4, DS5, 207, 207 CC, 207 SW']},
 {'Name': 'für Modell-4',
  'Value': ['208 I,

In [18]:
df_temp = pd.DataFrame([{'Item_Number': list_item[i],
                         'Title': title,
                         'Price': price,
                         'Sold': sold,
                         'Vehicle': vehicle,
                         'Src': src,
                         'status': 'ok'}])

df_temp

,Item_Number,Title,Price,Sold,Vehicle,Src,status
0,265482076619,Ventildeckel Zylinderkopfhaube für Citroën C4 ...,EUR 74.99,21,Citroën C4 Coupe -\nCitroën C4 Grand Picasso I...,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/2YkA...,ok


In [19]:
for name_value in details:
    df_temp[name_value['Name'].lower()] = ';'.join(name_value['Value'])
    
df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,Item_Number,Title,Price,Sold,Vehicle,Src,status,hersteller,produktart,herstellernummer,...,produktgruppe,kompatibel für,für modell-1,für modell-2,für modell-3,für modell-4,für modell-5,kraftstoff,ergänzungsartikel,lieferumfang
0,265482076619,Ventildeckel Zylinderkopfhaube für Citroën C4 ...,EUR 74.99,21,Citroën C4 Coupe -\nCitroën C4 Grand Picasso I...,https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/2YkA...,ok,nicht zutreffend,"Motorventilabdeckung, Zylinderkopfabdeckung, M...",nicht zutreffend,...,"Ventildeckeldichtung, Ventildeckel, Zylinderko...","Citroën, Peugeot","C4 Coupe, C4 Grand Picasso I, C4 Grand Picasso...","C4 Picasso I, C4 Picasso II, C5 III, C5 III Break","DS3, DS3 Cabriolet, DS4, DS5, 207, 207 CC, 207 SW","208 I, 3008 Großraumlimousine, 308 CC, 308 I, ...","308 SW II, 5008, 508 I, 508 SW I, RCZ",Benzin,mit Dichtung,1x Zylinderkopfhaube
